## Import libraries 

In [2]:
import pandas as pd 
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack

Read Excel with Pandas library 

In [3]:
df = pd.read_excel("datasets/final/dataset_linkedin.xlsx")
df.head()

,subject,body,from,date,label
0,"Julien, votre candidature a été envoyée à XRAY...",Votre candidature a été envoyée à XRAYS TRADIN...,jobs-noreply@linkedin.com,16/11/2025 10:57:31,1
1,"Julien, votre candidature a été envoyée à Equancy",Votre candidature a été envoyée à Equancy\n\n\...,jobs-noreply@linkedin.com,12/11/2025 13:55:57,1
2,"Julien, votre candidature a été envoyée à Inno...",Votre candidature a été envoyée à Innova Solut...,jobs-noreply@linkedin.com,12/11/2025 13:55:23,1
3,"Julien, votre candidature a été envoyée à Aude...",Votre candidature a été envoyée à Audensiel\n\...,jobs-noreply@linkedin.com,12/11/2025 13:54:50,1
4,"Julien, votre candidature a été envoyée à Extia",Votre candidature a été envoyée à Extia\n\n\n\...,jobs-noreply@linkedin.com,12/11/2025 13:53:25,1


## Basics EDA (appropriate in this context)

### 1. Analyzing the data  

In [4]:
df.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 195 entries, 0 to 194
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   subject  195 non-null    object
 1   body     195 non-null    object
 2   from     195 non-null    object
 3   date     195 non-null    object
 4   label    195 non-null    int64 
dtypes: int64(1), object(4)
memory usage: 7.7+ KB


### 2. Checking missing values 

In [5]:
df.isnull().sum()

subject    0
body       0
from       0
date       0
label      0
dtype: int64

### 3. Checking for the duplicates values 

In [6]:
df.nunique()

subject    137
body       195
from         1
date       195
label        4
dtype: int64

The last result let us understand that even if there is several labels, there is only one mail that is sending messages. Let's verify which one is it.

In [7]:
print(*set(df["from"])) # Set = drop duplicates, * = unpack the set 

jobs-noreply@linkedin.com


<b>FIRST RULE : VERIFY THAT THE MAIL IS COMING FROM jobs-noreply@linkedin.com TO ANALYSE IT</b>

## I wanna see for each label, which words is the most common in the subject :  

In [8]:
print(set(df["label"])) # all the label values

{0, 1, 4, 5}


In [9]:
def get_words_by_label_in_subject(df, label_value):
    return (
        df.loc[df["label"] == label_value, "subject"]
        .str.lower()
        .str.split()
        .explode()
    )

### number element labelised 0

In [10]:
nb_label0 = len(list(df.loc[df["label"] ==0, "subject"]))
print(nb_label0)

68


In [11]:
words = get_words_by_label_in_subject(df,0)
words.value_counts().head(10)

julien,         49
nouvelles       49
vous            34
chez            33
cette           30
semaine         30
des             29
avez            29
candidatures    29
d’emploi        22
Name: subject, dtype: int64

when the email has nothing to do with it, there is no keyword that stand out 

### number element labelised 1 (rule defined ✅)

In [12]:
nb_label1 = len(list(df.loc[df["label"] ==1, "subject"]))
print(nb_label1)

92


In [13]:
words = get_words_by_label_in_subject(df,1)

words.value_counts()[lambda x: x == nb_label1]

julien,        92
candidature    92
a              92
été            92
envoyée        92
à              92
votre          92
Name: subject, dtype: int64

In [14]:
df.loc[df["label"] ==1, "subject"].head()

0    Julien, votre candidature a été envoyée à XRAY...
1    Julien, votre candidature a été envoyée à Equancy
2    Julien, votre candidature a été envoyée à Inno...
3    Julien, votre candidature a été envoyée à Aude...
4      Julien, votre candidature a été envoyée à Extia
Name: subject, dtype: object

The last results let us understand that the pattern : <b>"Prénom, votre candidature a été envoyée à"</b> appear for each element labelised 1 (see the word document to remind the labels meaning)

<b> SECOND RULE : VERIFY HOW IS STRUCTURED THE SUBJECT MESSAGE, IF -> "Name, votre candidature a été envoyée à" IT MEANS THAT THE COMPANY JUST RECEIVE THE APPLICATION </b>

### number element labelised 4 (rule defined ✅)

In [15]:
nb_label4 = len(list(df.loc[df["label"] ==4, "subject"]))
print(nb_label4)

12


In [16]:
words = get_words_by_label_in_subject(df,4)

words.value_counts()[lambda x: x == nb_label4]

votre          12
a              12
été            12
vue            12
par            12
candidature    12
Name: subject, dtype: int64

In [17]:
df.loc[df["label"] ==4, "subject"].head()

93         Votre candidature a été vue par XRAYS TRADING
100                Votre candidature a été vue par ALTEN
128                Votre candidature a été vue par Extia
130            Votre candidature a été vue par MR SEARCH
132    Votre candidature a été vue par DGSE - Directi...
Name: subject, dtype: object

The last results let us understand that the pattern : <b>"votre candidature a été vue par"</b> appear for each element labelised 4 (see the word document to remind the labels meaning)

<b> THIRD RULE : VERIFY HOW IS STRUCTURED THE SUBJECT MESSAGE, IF -> "votre candidature a été vue par" IT MEANS THAT THE COMPANY JUST SAW YOUR APPLICATION (POSITIVE WITHOUT ACTION) </b>

### number labelised 5 (rule defined ✅)

In [18]:
nb_label5 = len(list(df.loc[df["label"] ==5, "subject"]))
print(nb_label5)

23


In [19]:
words = get_words_by_label_in_subject(df,5)

words.value_counts()[lambda x: x == nb_label5]

votre          23
:              23
chez           23
candidature    23
Name: subject, dtype: int64

In [20]:
df.loc[df["label"] ==5, "subject"].head()

92     Votre candidature : Développeur SQL/Développeu...
95     Votre candidature : Data Engineer H/F chez Gro...
96           Votre candidature : Data Engineer chez CBTW
99     Votre candidature : Ingénieur Data PL/SQL - Co...
101    Votre candidature : Data engineer (F/H) chez B...
Name: subject, dtype: object

The last results let us understand that the pattern : <b>"Votre candidature : POSTE chez"</b> appear for each element labelised 5 (see the word document to remind the labels meaning)

<b> FOURTH RULE : VERIFY HOW IS STRUCTURED THE SUBJECT MESSAGE, IF -> "Votre candidature : [...] chez " IT MEANS THAT THE COMPANY DECLINED YOUR APPLICATION </b>

### REVIEW (ALL RULES) 

<b>FIRST RULE : VERIFY THAT THE MAIL IS COMING FROM jobs-noreply@linkedin.com TO ANALYSE IT</b> <br>

<b> SECOND RULE : VERIFY HOW IS STRUCTURED THE SUBJECT MESSAGE, IF -> "Name, votre candidature a été envoyée à" IT MEANS THAT THE COMPANY JUST RECEIVE THE APPLICATION </b> <br>

<b> THIRD RULE : VERIFY HOW IS STRUCTURED THE SUBJECT MESSAGE, IF -> "votre candidature a été vue par" IT MEANS THAT THE COMPANY JUST SAW YOUR APPLICATION (POSITIVE WITHOUT ACTION) </b> <br>

<b> FOURTH RULE : VERIFY HOW IS STRUCTURED THE SUBJECT MESSAGE, IF -> "Votre candidature : [...] chez " IT MEANS THAT THE COMPANY DECLINED YOUR APPLICATION </b>

## Now, let's focus on the information that we can extract for the applicant
reference in the URL, company name, position, localisation, link toward the position

### When mail = application received

In [21]:
# Company name
df.loc[df["label"] ==1, "subject"].str.extract(r"à\s*(.*)")

,0
0,XRAYS TRADING
1,Equancy
2,Innova Solutions
3,Audensiel
4,Extia
...,...
87,act digital
88,moOngy Digital Lab
89,SERMA Safety and Security
90,CLEMLAB


### when mail = seen 

In [23]:
# company name
df.loc[df["label"] ==4, "subject"].str.extract(r"par\s*(.*)") # drop_duplicates()

,0
93,XRAYS TRADING
100,ALTEN
128,Extia
130,MR SEARCH
132,DGSE - Direction Générale de la Sécurité Extér...
140,DEAPDATA
141,Beelix
156,ACSSI
161,DGSI - Direction Générale de la Sécurité Intér...
171,Aspire Life Sciences Search


In [30]:
# link job
df["first_job_link"] = (
    df.loc[df["label"] ==4, "body"]
      .str.extract(r"(https://www\.linkedin\.com/comm/jobs/view/[^\s]+)")
)
df

,subject,body,from,date,label,first_job_link
0,"Julien, votre candidature a été envoyée à XRAY...",Votre candidature a été envoyée à XRAYS TRADIN...,jobs-noreply@linkedin.com,16/11/2025 10:57:31,1,NaN
1,"Julien, votre candidature a été envoyée à Equancy",Votre candidature a été envoyée à Equancy\n\n\...,jobs-noreply@linkedin.com,12/11/2025 13:55:57,1,NaN
2,"Julien, votre candidature a été envoyée à Inno...",Votre candidature a été envoyée à Innova Solut...,jobs-noreply@linkedin.com,12/11/2025 13:55:23,1,NaN
3,"Julien, votre candidature a été envoyée à Aude...",Votre candidature a été envoyée à Audensiel\n\...,jobs-noreply@linkedin.com,12/11/2025 13:54:50,1,NaN
4,"Julien, votre candidature a été envoyée à Extia",Votre candidature a été envoyée à Extia\n\n\n\...,jobs-noreply@linkedin.com,12/11/2025 13:53:25,1,NaN
...,...,...,...,...,...,...
190,"Julien, vous cherchez un nouveau poste ?",Laissez les opportunités professionnelles veni...,jobs-noreply@linkedin.com,2022-05-13 22:14:31,0,NaN
191,Nouvelles offres d’emploi similaires à [ALTERN...,Offres d’emploi comme celle que vous avez vue\...,jobs-noreply@linkedin.com,2022-05-07 22:51:44,0,NaN
192,Nouvelles offres d’emploi similaires à Stagiai...,Offres d’emploi comme celle que vous avez vue\...,jobs-noreply@linkedin.com,2022-04-18 20:27:38,0,NaN
193,"Julien, vous cherchez un nouveau poste ?",Laissez les opportunités professionnelles veni...,jobs-noreply@linkedin.com,2022-02-11 19:43:47,0,NaN


### when mail = declined

In [26]:
# company name
df.loc[df["label"] ==5, "subject"].str.extract(r"chez\s*(.*)") # drop_duplicates()

,0
92,Innova Solutions
95,Groupe EDH
96,CBTW
99,IKA
101,Beelix
106,Doctolib
134,Orange Business
136,Innova Solutions
137,DEAPDATA
142,AS International Group


In [27]:
# link job
df["first_job_link"] = (
    df.loc[df["label"] ==5, "body"]
      .str.extract(r"(https://www\.linkedin\.com/comm/jobs/view/[^\s]+)")
)


In [53]:
df.loc[df["label"] ==5]["body"][92]

'Dernière nouvelle de Innova Solutions\n\n\n\n----------------------------------------\n\n\n\nCet e-mail est destiné à Julien Ohana (Ingénieur Data | Ex. Thales)\n\nDécouvrez pourquoi nous précisons ceci\xa0: https://www.linkedin.com/help/linkedin/answer/4788?lang=fr&lipi=urn%3Ali%3Apage%3Aemail_email_jobs_application_rejected_01%3BIzuM9zfmRZSHgP%2Fbf6PrMA%3D%3D&midToken=AQHMKonoCAnG1w&midSig=2dFdtFflKitc01&trk=eml-email_jobs_application_rejected_01-SecurityHelp-0-textfooterglimmer&trkEmail=eml-email_jobs_application_rejected_01-SecurityHelp-0-textfooterglimmer-null-fqd7q9~mi3dskl8~ao-null-null&eid=fqd7q9-mi3dskl8-ao&otpToken=MWIwMDFmZTIxNTI3Y2JjNGI1MjQwNGVkNDAxZGU0YmQ4N2NjZDU0OTkxYTk4YjYxNzljNTA5NmI0ODVkNTRmNmY3ZDJkZmI5NmZlMmMzZmM2NTgwZDMwMTNiMDM3MDk2YjVkZmQ4NjczZDI5NWZkMGRhYTljMiwxLDE%3D\n\nVous recevez des e-mails de notifications de LinkedIn.\n\n\n\n\n\nSe désabonner\xa0: https://www.linkedin.com/comm/psettings/email-unsubscribe?lipi=urn%3Ali%3Apage%3Aemail_email_jobs_application_r

## MAUVAISE Récupération dans le body des mails === 5 